In [1]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib import style
import csv

In [2]:
filename = ['08.01.2021 100929', '08.01.2021 104013', '08.01.2021 112923', '08.01.2021 134757']
TimeShift = [0, 0, 0, 0] # Shift time to match with other data. In (ms)
experiment = ['DXambient', 'SXambient', 'SXcold', 'DXcold']
bootname = ['1-Left', '1-Left', '1-Left', '1-Left']
username = ['User 1', 'User 1', 'User 1', 'User 1']

In [3]:
appendList = []
col_names = ['Time','AccX','AccY','AccZ','GyroX','GyroY','GyroZ','MagnX','MagnY','MagnZ','P0','P1','P2','P3','P4','P5','P6','P7','Temperature']

for i in range(len(filename)):
    # Read in the .txt -files and insert the column-names
    ReadFile = pd.read_csv ('Files/'+filename[i]+'.txt', delimiter='\t', skiprows=9)
    ReadFile.columns=col_names
    # Change timestamp to date and time
    ReadFile.Time = pd.to_datetime(ReadFile.Time + TimeShift[i], unit='ms')
    # Insert additional columns
    ReadFile.insert(1, 'Experiment', experiment[i])
    ReadFile.insert(2, 'Boot', bootname[i])
    ReadFile.insert(3, 'User', username[i])
    # Append all files together
    appendList.append(ReadFile)

# Concatenate list of appended files
DataFile = pd.concat(appendList)

In [4]:
data = DataFile.to_csv (r'testtext2.csv', index=None)

In [5]:
# Connect with database and create curser
con = sqlite3.connect('testDB.db')
cur = con.cursor()
# Set name of table
tablename = 'test333'
# Create a new SQL-Table if none exists, with name set in  tablename
# Types in SQLite: str -> TEXT , int -> INTEGER , float -> REAL
cur.execute("""CREATE TABLE IF NOT EXISTS {}(Time TEXT, Experiment TEXT, Boot TEXT, User TEXT,AccX REAL, AccY REAL,
                                             AccZ REAL, GyroX REAL, GyroY REAL,GyroZ REAL, MagnX REAL, MagnY REAL,
                                             MagnZ REAL, P0 REAL,P1 REAL, P2 REAL, P3 REAL, P4 REAL, P5 REAL, P6 REAL,
                                             P7 REAL, Temperature REAL)""".format('table_name'))
# Save (commit) the changes
con.commit()
# Open the .csv -file and lode it into DataFrame
df = pd.read_csv('testtext2.csv', delimiter=',')
# Insert DataFrame to SQL-Table
df.to_sql(tablename, con, if_exists='replace', index=False)
# Securely close the connection
con.close()